In [8]:
import os
import glob
import xml.etree.ElementTree as ET

# Define paths
ANNOTATIONS_PATH = "annotations/train"  # Change to "annotations/test" for test set
LABELS_PATH = "labels/train"  # Change to "labels/test" for test set
os.makedirs(LABELS_PATH, exist_ok=True)

# Define class labels
CLASS_NAMES = ["ThumbsUp", "ThumbsDown", "ThankYou", "LiveLong"]  # Update with your class names

# Function to convert XML to YOLO
def convert_voc_to_yolo(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    img_width = int(root.find("size/width").text)
    img_height = int(root.find("size/height").text)

    filename = os.path.splitext(root.find("filename").text)[0]  # Remove .jpg/.png
    label_file_path = os.path.join(LABELS_PATH, f"{filename}.txt")

    with open(label_file_path, "w") as label_file:
        for obj in root.findall("object"):
            class_name = obj.find("name").text
            if class_name not in CLASS_NAMES:
                continue  # Skip if class is not in list
            
            class_id = CLASS_NAMES.index(class_name)  # Get class index
            bbox = obj.find("bndbox")
            xmin = int(bbox.find("xmin").text)
            ymin = int(bbox.find("ymin").text)
            xmax = int(bbox.find("xmax").text)
            ymax = int(bbox.find("ymax").text)

            # Convert to YOLO format (x_center, y_center, width, height) normalized
            x_center = (xmin + xmax) / (2 * img_width)
            y_center = (ymin + ymax) / (2 * img_height)
            width = (xmax - xmin) / img_width
            height = (ymax - ymin) / img_height

            label_file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

# Convert all XML files
xml_files = glob.glob(os.path.join(ANNOTATIONS_PATH, "*.xml"))
for xml_file in xml_files:
    convert_voc_to_yolo(xml_file)

print("Conversion complete! Check 'labels/train/' for YOLO TXT files.")


Conversion complete! Check 'labels/train/' for YOLO TXT files.


In [9]:
from ultralytics import YOLO
print("YOLOv8 is ready!")


YOLOv8 is ready!


In [10]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

results = model.train(
    data="dataset.yaml",  
    epochs=50,  
    imgsz=640,  
    batch=8,  
    workers=2,  
    device="cpu"
)


Ultralytics 8.3.82  Python-3.9.21 torch-2.6.0+cpu CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=2, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxe

100%|███████████████████████████████████████████████████████████████████████████████| 755k/755k [00:00<00:00, 6.65MB/s]

Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     


  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  8                  -1  1    460288  ultralytics.nn.modules.block.C2f             [256,

train: Scanning C:\Users\varun\workspace\labels\train... 15 images, 0 backgrounds, 0 corrupt: 100%|██████████| 15/15 [0

...

      50/50         0G     0.4375      1.545     0.9009          7        640: 100%|██████████| 2/2 [00:05<00:00,  2.60
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<0

                   all          4          4      0.888          1      0.995      0.895



50 epochs completed in 0.109 hours.
Optimizer stripped from runs\detect\train4\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train4\weights\best.pt, 6.2MB

Validating runs\detect\train4\weights\best.pt...
Ultralytics 8.3.82  Python-3.9.21 torch-2.6.0+cpu CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<0


                   all          4          4      0.887          1      0.995      0.895
              ThumbsUp          1          1      0.947          1      0.995      0.796
            ThumbsDown          1          1      0.938          1      0.995      0.995
              ThankYou          1          1      0.854          1      0.995      0.895
              LiveLong          1          1      0.811          1      0.995      0.895
Speed: 1.5ms preprocess, 75.6ms inference, 0.0ms loss, 5.1ms postprocess per image
Results saved to runs\detect\train4


In [17]:
from ultralytics import YOLO

model = YOLO("C:/Users/varun/workspace/runs/detect/train4/weights/best.pt")  # Load the best model
metrics = model.val()  # Validate on test dataset


Ultralytics 8.3.82  Python-3.9.21 torch-2.6.0+cpu CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\varun\workspace\labels\test.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [0
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<0


                   all          4          4      0.887          1      0.995      0.895
              ThumbsUp          1          1      0.947          1      0.995      0.796
            ThumbsDown          1          1      0.938          1      0.995      0.995
              ThankYou          1          1      0.854          1      0.995      0.895
              LiveLong          1          1      0.811          1      0.995      0.895
Speed: 1.9ms preprocess, 67.5ms inference, 0.0ms loss, 7.2ms postprocess per image
Results saved to runs\detect\val


In [3]:

model.predict(source=0, show=True, conf=0.5)



1/1: 0... Success  (inf frames of shape 640x480 at 30.00 FPS)


WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

0: 480x640 (no detections), 204.6ms
0: 480x640 (no detections), 80.5ms
0: 480x640 (no detections), 66.0ms
0: 480x640 (no detections), 59.3ms
0: 480x640 (no detections), 51.6ms
WARNING  Waiting for stream 0
0: 480x640 (no detections), 52.4ms
0: 480x640 (no detections), 61.8ms
0: 480x640 (no detections), 66.6ms
0: 480x640 (no detections), 66.6ms
0: 480x640 (no detections), 64.4ms
0: 480x640 (no

KeyboardInterrupt: 

In [2]:
from ultralytics import YOLO

# Load your trained model
model = YOLO("C:/Users/varun/workspace/runs/detect/train4/weights/best.pt")  

# Run validation on the test dataset
metrics = model.val()

# Print mAP results
print(metrics)


Ultralytics 8.3.82  Python-3.9.21 torch-2.6.0+cpu CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\varun\workspace\labels\test.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [0
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<0


                   all          4          4      0.887          1      0.995      0.895
              ThumbsUp          1          1      0.947          1      0.995      0.796
            ThumbsDown          1          1      0.938          1      0.995      0.995
              ThankYou          1          1      0.854          1      0.995      0.895
              LiveLong          1          1      0.811          1      0.995      0.895
Speed: 2.4ms preprocess, 58.4ms inference, 0.0ms loss, 3.7ms postprocess per image
Results saved to runs\detect\val2
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001B97B831370>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005